In [ ]:
# import the library
import tensorflow as tf  
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras.optimizers import SGD, Adam
from keras import callbacks
from sklearn.model_selection import train_test_split

In [ ]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10
 
# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

170508288/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [ ]:
# Multi-label Encoding
def  multi_label_encoding(y):
  animals = [ 3, 4, 5, 6, 7]
  vehicles = [0, 1, 8, 9]
  fly = [0,2]
  walk = [2, 3, 4, 5, 9]
  jump = [3, 4, 5, 6, 7]
  swim = [3, 4, 6, 7, 8 ]
  new_y = np.zeros((len(y), 6), dtype=np.uint8)
  copy_y= y.copy()
  for i in range(0, len(y)):
    if copy_y[i] in animals:
      new_y[i][0] = 1
    if copy_y[i] in vehicles:
      new_y[i][1] = 1
    if copy_y[i] in fly:
      new_y[i][2] = 1
    if copy_y[i] in walk:
      new_y[i][3] = 1
    if copy_y[i] in jump:
      new_y[i][4] = 1
    if copy_y[i] in swim:
      new_y[i][5] = 1
  return  new_y

def scale_pixel(x):
  # convert from integers to floats
  new_x = x.astype('float32')
  # normalize to range 0-1
  new_x = new_x / 255.0
  return new_x

Y_train = multi_label_encoding(y_train)
y_test = multi_label_encoding(y_test)
X_train = scale_pixel(x_train)
x_test = scale_pixel(x_test)
x_val, x_train,y_val,y_train= train_test_split(X_train, Y_train, test_size = 0.8)

In [ ]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.2))
	model.add(Dense(6, activation='sigmoid'))
	# compile model
	opt = SGD(learning_rate=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

def summarize_diagnostics(history):
  # plot loss
  pyplot.subplot(211)
  pyplot.title('Cross Entropy Loss')
  pyplot.plot(history.history['loss'], color='blue', label='train')
  pyplot.plot(history.history['val_loss'], color='orange', label='test')
  # plot accuracy
  pyplot.subplot(212)
  pyplot.title('Classification Accuracy')
  pyplot.plot(history.history['accuracy'], color='blue', label='train')
  pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
  # save plot to file
  filename = sys.argv[0].split('/')[-1]
  pyplot.savefig(filename + '_plot.png')
  pyplot.close()

In [ ]:
# run the test harness for evaluating a model
def run_test_harness():
  model = define_model()
	# fit model
  history = model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test), verbose=0)
	# evaluate model
  _, acc = model.evaluate(x_test, y_test, verbose=0)
  print('> %.3f' % (acc * 100.0))
	# learning curves
  summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

> 87.870


# New Section